In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 48 kB/s 
     |████████████████████████████████| 199 kB 64.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=91ba726ffd2ad93337c240d6b630915be3dc45fb49037d698718d0a3c91d5c7b
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import random
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType, StructType
from pyspark.sql.functions import split
from pyspark.sql.functions import *
from pyspark.sql.functions import window


In [ ]:
spark = SparkSession\
.builder\
.appName('Project2')\
.master("local[*]")\
.config("spark.executor.memory", "70g")\
.config("spark.driver.memory", "50g")\
.config("spark.memory.offHeap.enabled",True)\
.config("spark.memory.offHeap.size","16g")\
.appName("sampleCodeForReference")\
.getOrCreate()

In [ ]:
df = spark.read.parquet("/content/drive/MyDrive/springboard2")

In [ ]:
ym_list = df\
.withColumn('ym',F.col("date").substr(1,6))\
.select("ym").distinct()\
.toPandas()['ym']

subset = []
for local_ym in ym_list:
  df_subset = df\
  .withColumn('ym',F.col("date").substr(1,6))\
  .filter(F.col("ym") == local_ym)
  subset.append(df_subset)


In [ ]:
df.printSchema()

root
 |-- visitorId: long (nullable = true)
 |-- visitNumber: long (nullable = true)
 |-- visitId: long (nullable = true)
 |-- visitStartTime: long (nullable = true)
 |-- date: string (nullable = true)
 |-- totals: struct (nullable = true)
 |    |-- visits: long (nullable = true)
 |    |-- hits: long (nullable = true)
 |    |-- pageviews: long (nullable = true)
 |    |-- timeOnSite: long (nullable = true)
 |    |-- bounces: long (nullable = true)
 |    |-- transactions: long (nullable = true)
 |    |-- transactionRevenue: long (nullable = true)
 |    |-- newVisits: long (nullable = true)
 |    |-- screenviews: long (nullable = true)
 |    |-- uniqueScreenviews: long (nullable = true)
 |    |-- timeOnScreen: long (nullable = true)
 |    |-- totalTransactionRevenue: long (nullable = true)
 |    |-- sessionQualityDim: long (nullable = true)
 |-- trafficSource: struct (nullable = true)
 |    |-- referralPath: string (nullable = true)
 |    |-- campaign: string (nullable = true)
 |    |-- s

In [ ]:
def flatten_df(nested_df):
    stack = [((), nested_df)]
    columns = []

    while len(stack) > 0:
        parents, df = stack.pop()

        flat_cols = [
            col(".".join(parents + (c[0],))).alias("_".join(parents + (c[0],)))
            for c in df.dtypes
            if c[1].startswith("struct") == False
        ]

        nested_cols = [
            c[0]
            for c in df.dtypes
            if c[1].startswith("struct")
        ]

        columns.extend(flat_cols)

        for nested_col in nested_cols:
            projected_df = df.select(nested_col + ".*")
            stack.append((parents + (nested_col,), projected_df))

    return nested_df.select(columns)

def unpack(df):
  array_cols = [c[0] for c in df.dtypes if c[1].startswith("array")]
  for c in array_cols:
    df = df.withColumn("_".join((c, "array")), F.col(c)[0]).drop(c)
  return df

def combine(df):
  cols = ["dummy"]
  while len(cols) >0:
    df = unpack(df)
    df = flatten_df(df)
    cols = [c[0] for c in df.dtypes if c[1].startswith("struct")|c[1].startswith("array")]
  return df


In [ ]:
df = combine(df)

In [ ]:
df.show()

+---------+-----------+----------+--------------+--------+-------------------+------+--------+---------------+--------------------+--------------------+---------------+---------------+-----------------+-------------------+------------------------+---------------------+-----------------+--------------------+---------------+---------------------+-------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+---------------------------------------------+---------------------------------------------+---------------------------------------------+---------------------------------------------+---------------------------------------------+------------------------------------------------+------------------------------------------------+------------------------------------------------+------------------------------------------------+--------------------------------------

In [ ]:
#Flatten each file (ym) and add them to df_list 
import pickle
flat_df = combine(subset[0])
pandas_df = flat_df.toPandas()
path = '/content/drive/My Drive/capstone 2 pickle files/file1.pickle'
with open(path, 'wb') as f:
  pickle.dump(pandas_df, f)

In [ ]:
import pandas as pd
# df1 = pd.read_pickle('/content/drive/My Drive/capstone 2 pickle files/file1.pickle')
# df2 = pd.read_pickle('/content/drive/My Drive/capstone 2 pickle files/file2.pickle')
# df3 = pd.read_pickle('/content/drive/My Drive/capstone 2 pickle files/file3.pickle')
# df4 = pd.read_pickle('/content/drive/My Drive/capstone 2 pickle files/file4.pickle')
# df5 = pd.read_pickle('/content/drive/My Drive/capstone 2 pickle files/file5.pickle')
# df6 = pd.read_pickle('/content/drive/My Drive/capstone 2 pickle files/file6.pickle')
# df7 = pd.read_pickle('/content/drive/My Drive/capstone 2 pickle files/file7.pickle')
# df8 = pd.read_pickle('/content/drive/My Drive/capstone 2 pickle files/file8.pickle')
# df9 = pd.read_pickle('/content/drive/My Drive/capstone 2 pickle files/file9.pickle')
# df10 = pd.read_pickle('/content/drive/My Drive/capstone 2 pickle files/file10.pickle')
# df11 = pd.read_pickle('/content/drive/My Drive/capstone 2 pickle files/file11.pickle')
# df12 = pd.read_pickle('/content/drive/My Drive/capstone 2 pickle files/file12.pickle')
# df13 = pd.read_pickle('/content/drive/My Drive/capstone 2 pickle files/file13.pickle')

# df123 = pd.read_pickle('/content/drive/My Drive/capstone 2 pickle files/combine123.pickle')
# df456 = pd.read_pickle('/content/drive/My Drive/capstone 2 pickle files/combine456.pickle')
# df789 = pd.read_pickle('/content/drive/My Drive/capstone 2 pickle files/combine789.pickle')
# df10111213 = pd.read_pickle('/content/drive/My Drive/capstone 2 pickle files/combine10111213.pickle')
df_1_6 = pd.read_pickle('/content/drive/My Drive/capstone 2 pickle files/combine_1_6.pickle')
df_7_13 = pd.read_pickle('/content/drive/My Drive/capstone 2 pickle files/combine_7_13.pickle')

In [ ]:
import pickle
combine_all = pd.concat([df_1_6, df_7_13], ignore_index = True)
path = '/content/drive/My Drive/capstone 2 pickle files/combine_all.pickle'
with open(path, 'wb') as f:
  pickle.dump(combine_all, f)

In [ ]:
import pandas as pd
df = pd.read_pickle('/content/drive/My Drive/capstone 2 pickle files/combine_all.pickle')


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Columns: 307 entries, visitorId to hits_array_product_array_customDimensions_array_value
dtypes: bool(3), float64(155), int64(10), object(139)
memory usage: 2.0+ GB


In [ ]:
missing = pd.concat([df.isnull().sum(), 100 * df.isnull().mean()], axis = 1)
missing.columns = ['count','%']
missing.sort_values(by = '%', ascending = False)

,count,%
visitorId,903653,100.0
totals_uniqueScreenviews,903653,100.0
hits_array_item_localItemRevenue,903653,100.0
hits_array_transaction_transactionCoupon,903653,100.0
device_javaEnabled,903653,100.0
...,...,...
device_browserSize,0,0.0
device_operatingSystem,0,0.0
device_operatingSystemVersion,0,0.0
device_isMobile,0,0.0


In [ ]:
missing_all = missing[missing["%"] == 100]
missing_all.index

Index(['visitorId', 'userId', 'clientId', 'hits_array_isSecure',
       'hits_array_contentGroup_contentGroupUniqueViews4',
       'hits_array_contentGroup_contentGroupUniqueViews5',
       'hits_array_sourcePropertyInfo_sourcePropertyDisplayName',
       'hits_array_sourcePropertyInfo_sourcePropertyTrackingId',
       'hits_array_latencyTracking_userTimingValue',
       'hits_array_latencyTracking_userTimingSample',
       ...
       'hits_array_customVariables_array_customVarValue',
       'hits_array_experiment_array_experimentId',
       'hits_array_experiment_array_experimentVariant',
       'hits_array_product_array_productRefundAmount',
       'hits_array_product_array_localProductRefundAmount',
       'hits_array_product_array_productCouponCode',
       'hits_array_product_array_customMetrics_array_index',
       'hits_array_product_array_customMetrics_array_value',
       'hits_array_product_array_customDimensions_array_index',
       'hits_array_product_array_customDimensions

In [ ]:
df.drop(missing_all.index, axis = 1, inplace = True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Columns: 149 entries, visitNumber to hits_array_product_array_productListPosition
dtypes: bool(3), float64(36), int64(10), object(100)
memory usage: 1009.2+ MB


In [ ]:
import pickle

path = '/content/drive/My Drive/capstone 2 pickle files/cleaned_file.pickle'
with open(path, 'wb') as f:
  pickle.dump(df, f)